# Talks markdown generator for academicpages

Takes a TSV of talks with metadata and converts them for use with [academicpages.github.io](academicpages.github.io). This is an interactive Jupyter notebook ([see more info here](http://jupyter-notebook-beginner-guide.readthedocs.io/en/latest/what_is_jupyter.html)). The core python code is also in `talks.py`. Run either from the `markdown_generator` folder after replacing `talks.tsv` with one containing your data.

TODO: Make this work with BibTex and other databases, rather than Stuart's non-standard TSV format and citation style.

In [23]:
import pandas as pd
import os

## Data format

The TSV needs to have the following columns: title, type, url_slug, venue, date, location, talk_url, description, with a header at the top. Many of these fields can be blank, but the columns must be in the TSV.

- Fields that cannot be blank: `title`, `url_slug`, `date`. All else can be blank. `type` defaults to "Talk" 
- `date` must be formatted as YYYY-MM-DD.
- `url_slug` will be the descriptive part of the .md file and the permalink URL for the page about the paper. 
    - The .md file will be `YYYY-MM-DD-[url_slug].md` and the permalink will be `https://[yourdomain]/talks/YYYY-MM-DD-[url_slug]`
    - The combination of `url_slug` and `date` must be unique, as it will be the basis for your filenames

This is how the raw file looks (it doesn't look pretty, use a spreadsheet or other program to edit and create).

In [24]:
!cat talks.tsv

'cat' is not recognized as an internal or external command,
operable program or batch file.


## Import TSV

Pandas makes this easy with the read_csv function. We are using a TSV, so we specify the separator as a tab, or `\t`.

I found it important to put this data in a tab-separated values format, because there are a lot of commas in this kind of data and comma-separated values can get messed up. However, you can modify the import statement, as pandas also has read_excel(), read_json(), and others.

In [25]:
talks = pd.read_csv("talks.tsv", sep="\t", header=0)
talks

title             type  \
0            Quantum counting and the symmetric group  Invited seminar   
1   Approximation algorithms for dense quantum Ham...  Invited seminar   
2   Quantum Hamiltonian Complexity in thermal equi...  Invited seminar   
3   Quantum simulation and complexity of thermal p...  Invited seminar   
4    Quantum computing for quantum physics and beyond  Invited seminar   
5   Quantum and classical algorithms for trace-est...  Conference talk   
6    On the complexity of quantum partition functions  Conference talk   
7   Complexity of approximating quantum many-body ...  Invited seminar   
8   Approximating quantum many-body physics in the...  Invited seminar   
9    On the complexity of quantum partition functions  Conference talk   
10  Approximating the free energy of dense quantum...  Invited seminar   
11  Approximating the free energy of dense quantum...  Invited seminar   
12  Classical and quantum algorithms for estimatin...     Invited talk   
13  Computing partition functions with one clean q...  Invited seminar   
14    Simulating thermal physics on quantum computers  Invited seminar   
15  Computing partition functions with one clean q...  Conference talk   
16    Simulating thermal physics on quantum computers  Conference talk   
17  Improved quantum simulation algorithms using a...  Invited seminar   
18    Improved implementation of reflection operators  Conference talk   
19  Improved quantum algorithms using linear combi...  Conference talk   
20     Quantum algorithms for hitting-time estimation  Conference talk   

       url_slug                                              venue  \
0       ipam_23     Institute of Pure and Applied Mathematics UCLA   
1        cno_23                             University of Waterloo   
2      iiser_23  Indian Institute of Science Education and Rese...   
3         vt_23                                      Virginia Tech   
4        ets_22                Ecole de Technologie (ETS) Montreal   
5     squint_22  Southwest Quantum Information and Technology W...   
6        tqc_22  Theory of Quantum Computation, Communication a...   
7        iff_22                The Institute for Fundamental Study   
8        tii_22                    Technology Innovation Institute   
9        qip_22                Quantum Information Processing 2022   
10      lanl_22                    Los Alamos National Laboratory    
11       unm_22                           University of New Mexico   
12      ipam_22  IPAM Workshop on Quantum Numerical Linear Algebra   
13   bristol_21                              University of Bristol   
14  waterloo_20                             University of Waterloo   
15       tqc_20  Theory of Quantum Computation, Communication a...   
16    intriq_19                                INTRIQ Fall meeting   
17      lanl_19                     Los Alamos National Laboratory   
18       aps_19            American Physical Society March Meeting   
19    squint_18  Southwest Quantum Information and Technology W...   
20       aps_16            American Physical Society March meeting   

          date                        location  \
0   2023-11-14       Los Angeles, USA (remote)   
1   2023-06-05                Waterloo, Canada   
2   2023-05-25                     Pune, India   
3   2023-02-27                 Blacksburg, USA   
4   2022-12-09                Montreal, Canada   
5   2022-10-21                   Berkeley, USA   
6   2022-07-12             Champaign, Illinois   
7   2022-03-25  Phitsanulok, Thailand (remote)   
8   2022-03-17                  Abu Dhabi, UAE   
9   2022-03-11                   Pasadena, USA   
10  2022-02-10        Los Alamos, USA (remote)   
11  2022-02-03       Albuquerque, USA (remote)   
12  2022-01-26       Los Angeles, USA (remote)   
13  2021-03-31            Bristol, UK (remote)   
14  2021-03-12                Waterloo, Canada   
15  2020-06-10           Riga, Latvia (remote)   
16  2019-11-11              

## Escape special characters

YAML is very picky about how it takes a valid string, so we are replacing single and double quotes (and ampersands) with their HTML encoded equivilents. This makes them look not so readable in raw format, but they are parsed and rendered nicely.

In [26]:
html_escape_table = {
    "&": "&amp;",
    '"': "&quot;",
    "'": "&apos;"
    }

def html_escape(text):
    if type(text) is str:
        return "".join(html_escape_table.get(c,c) for c in text)
    else:
        return "False"

## Creating the markdown files

This is where the heavy lifting is done. This loops through all the rows in the TSV dataframe, then starts to concatentate a big string (```md```) that contains the markdown for each type. It does the YAML metadata first, then does the description for the individual page.

In [27]:
loc_dict = {}

for row, item in talks.iterrows():
    
    md_filename = str(item.date) + "-" + item.url_slug + ".md"
    html_filename = str(item.date) + "-" + item.url_slug 
    year = item.date[:4]
    
    md = "---\ntitle: \""   + item.title + '"\n'
    md += "collection: talks" + "\n"
    
    if len(str(item.type)) > 3:
        md += 'type: "' + item.type + '"\n'
    else:
        md += 'type: "Talk"\n'
    
    md += "permalink: /talks/" + html_filename + "\n"
    
    if len(str(item.venue)) > 3:
        md += 'venue: "' + item.venue + '"\n'
        
    if len(str(item.location)) > 3:
        md += "date: " + str(item.date) + "\n"
    
    if len(str(item.location)) > 3:
        md += 'location: "' + str(item.location) + '"\n'

    if len(str(item.talk_url)) > 3:
        md += "talk_url: " + item.talk_url + "\n" 
        
    if len(str(item.description)) > 3:
        md += "description: " + html_escape(item.description) + "\n"

    if len(str(item.video_url)) > 3:
        md += "video_url: " + item.video_url + ")\n" 
           
    md += "---\n"
        
    md_filename = os.path.basename(md_filename)
    #print(md)
    
    with open("../_talks/" + md_filename, 'w') as f:
        f.write(md)

These files are in the talks directory, one directory below where we're working from.

In [9]:
!ls ../_talks

'ls' is not recognized as an internal or external command,
operable program or batch file.


In [7]:
!cat ../_talks/2013-03-01-tutorial-1.md

'cat' is not recognized as an internal or external command,
operable program or batch file.
